In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

if os.getcwd().split("/")[-1] == "sandbox":
    os.chdir("../")


## Estimate weights

In [2]:
from estimator.chains.weight_est import get_weight_estimator_chain
weight_est = get_weight_estimator_chain(language="en",verbose=True)

In [3]:
output = weight_est.run(
    """150 g red lentils
        1 large onion, 1 pepper
        2 cubes of vegetable bouillon
        3 cloves garlic, pressed
        1 can of chopped tomatoes,
        1 tin of tomato concentrate (140 g),
        1 can of coconut milk (400 ml),
        1 tbsp. lemon juice,
        1 tsp. sugar,
        ½ teaspoon chili powder,
        ½ teaspoon ground cumin,
        Possibly. 1/4 tsp. smoked paprika,
        A large handful of fresh basil, chopped
        (or 2 tsp dried basil)"""
)



> Entering new LLMChain chain...
Prompt after formatting:

Given a list of ingredients, estimate the weights in kilogram for each ingredient.
Explain your reasoning for the estimation of weights.

The following general weights can be used for estimation:

1 can = 400 g = 0.4 kg
1 bouillon cube = 4 g = 0.004 kg
1 large onion = 285 g = 0.285 kg
1 medium onion = 170 g = 0.170 kg
1 small onion = 115 g = 0.115 kg
1 bell pepper = 150 g = 0.150 kg
1 can tomato paste = 140 g = 0.140 kg
1 tablespoon/tbsp. = 15 g  = 0.015 kg
1 teaspoon/tsp. = 5 g = 0.005 kg
1 potato = 170 - 300 g = 0.170 - 0.300 kg
1 carrot = 100 g = 0.100 kg
1 lemon = 85 g = 0.085 kg
1 tortilla = 30 g = 0.030 kg
1 squash = 400 g = 0.400 kg
1 clove garlic = 0.004 kg
Handful of herbs (basil, oregano etc.) = 0.025 kg

Examples of a bunch/bnch of an ingredient - use them as a guideline:
1 bunch/bnch parsley = 50 g = 0.050 kg
1 bunch/bnch asparagus = 500 g = 0.500 kg
1 bunch of carrots = 750 g = 0.750 kg
1 bunch/bnch tomatoes = 50

In [4]:
from estimator.prompt_templates.weight_est_prompts import weight_output_parser

parsed_output = weight_output_parser.parse(output)

### Prepare items for co2 query if they are not negligeble

In [5]:
negligeble_threshold = 0.01  # eq. to 10 grams

co2_query_input = [item.ingredient for item in parsed_output.weight_estimates if item.weight_in_kg is not None and item.weight_in_kg > negligeble_threshold]
co2_query_input_str = str(co2_query_input)

## Query CO2 emissions

In [6]:
from estimator.chains.co2_sql import get_co2_sql_chain

co2_sql_chain = get_co2_sql_chain(language="en",verbose=True)

In [7]:
sql_output = co2_sql_chain.run(co2_query_input_str)



> Entering new SQLDatabaseChain chain...
['150 g red lentils', '1 large onion', '1 pepper', '3 cloves garlic, pressed', '1 can of chopped tomatoes', '1 tin of tomato concentrate (140 g)', '1 can of coconut milk (400 ml)', '1 tbsp. lemon juice', 'A large handful of fresh basil, chopped (or 2 tsp dried basil)']
SQLQuery:SELECT Name, Total_kg_CO2_eq_kg FROM dk_co2_emission WHERE
          Name LIKE '%tomato%' OR
          Name LIKE '%lentil%' OR
          Name LIKE '%onion%' OR
          Name LIKE '%pepper%' OR
          Name LIKE '%garlic%' OR
          Name LIKE '%coconut milk%' OR
          Name LIKE '%lemon%' OR
          Name LIKE '%basil%'
SQLResult: [('Pepper, sweet, red, raw', 1.02), ('Tomato, ripe, raw, origin unknown', 0.7), ('Tomato, dried', 1.95), ('Pepper, hot chili, raw', 1.02), ('Tuna, in tomato sauce, canned', 9.15), ('Garlic, raw', 1.33), ('Ice, popsickle, lemonade', 1.15), ('Green lentils, dried', 1.78), ('Soup, tomato, ready to eat', 1.84), ('Onions, chopped, frozen',

In [8]:
from estimator.prompt_templates.co2_sql_prompts import sql_co2_output_parser

parsed_sql_output = sql_co2_output_parser.parse(sql_output)

In [9]:
for item in parsed_sql_output.emissions:
    print(item)

ingredient='150 g red lentils' comment='' unit='kg CO2e / kg' co2_per_kg=1.78
ingredient='1 large onion' comment='closest was Onion, raw' unit='kg CO2e / kg' co2_per_kg=0.9
ingredient='1 pepper' comment='closest was Pepper, sweet, red, raw' unit='kg CO2e / kg' co2_per_kg=1.02
ingredient='3 cloves garlic, pressed' comment='closest was Garlic, raw' unit='kg CO2e / kg' co2_per_kg=1.33
ingredient='1 can of chopped tomatoes' comment='closest was Tomato, peeled, canned' unit='kg CO2e / kg' co2_per_kg=1.26
ingredient='1 tin of tomato concentrate (140 g)' comment='closest was Tomato paste, concentrated' unit='kg CO2e / kg' co2_per_kg=2.48
ingredient='1 can of coconut milk (400 ml)' comment='closest was Coconut milk' unit='kg CO2e / kg' co2_per_kg=3.5
ingredient='1 tbsp. lemon juice' comment='closest was Lemon, raw' unit='kg CO2e / kg' co2_per_kg=0.94
ingredient='A large handful of fresh basil, chopped (or 2 tsp dried basil)' comment='closest was Basil, fresh' unit='kg CO2e / kg' co2_per_kg=0.3

## Get items with no results to use search on the net

In [10]:
co2_search_input_items = [item.ingredient for item in parsed_sql_output.emissions if item.co2_per_kg is None]


if not co2_search_input_items:
    co2_search_input_items = ["3 gulerødder", "en håndful basilikum", "3 tablespoons of olive oil"]

In [11]:
# import sys
# import os
# sys.path.append(os.path.dirname(os.getcwd()))

# if os.getcwd().split("/")[-1] == "sandbox":
#     os.chdir("../")

# from estimator.prompt_templates.weight_est_prompts import weight_output_parser
# from estimator.chains.weight_est import get_weight_est


In [12]:
from estimator.agent.search_agent import get_co2_google_search_agent
from estimator.prompt_templates.co2_search_prompts import search_co2_output_parser

search_agent = get_co2_google_search_agent(verbose=True)

co2_search_input_items_str = str(co2_search_input_items)

In [14]:
import asyncio
tasks = [search_agent.arun(q) for q in co2_search_input_items]
search_results = await asyncio.gather(*tasks)



> Entering new AgentExecutor chain...

> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Action: Search
Action Input: olive oil kg CO2e per kgAction: Search
Action Input: basilikum kg CO2e per kgAction: Search
Action Input: gulerødder kg CO2e per kg
Observation: Grøntsager, Gulerod, uspec., rå, 0,25, 0,11, 0,02, 0,00, 0,06, 0,05, 0,01. Grøntsager, Radise, rå, 0,37, 0,13, 0,03, 0,00, 0,14, 0,05, 0,01. Det samlede klimaaftryk per kg jordbær i supermarkedet bliver hhv. 0,91 og 0,88 kg CO2/kg jordbær, heraf antages der 0,260 g at stamme fra. Gulerødder er en af de mest klimavenlige madvarer der findes og så er de tilmed sunde! ... 0,25 kg CO2e pr. kg for lokale gulerødder i sæson 1 kg CO2e pr. I faktiske kg CO2e er udledningen fra emballage og transport i alle tilfælde ret lav. Emballagen ligger typisk på 50-100 g CO2e per kg ... I denne opskrift glaseres de søde gulerødder med syrlig appelsinjuice og serveres med estragonmayonnaise og sprøde friterede løg ..

In [15]:
parsed_search_results = [search_co2_output_parser.parse(result) for result in search_results]

print(parsed_search_results)


[CO2SearchResult(ingredient='3 gulerødder', explanation='The most likely result is chosen based on the range for vegetables.', unit='kg CO2e per kg', result=0.25), CO2SearchResult(ingredient='en håndful basilikum', explanation='CO2e per kg not found', unit=None, result=None), CO2SearchResult(ingredient='3 tablespoons of olive oil', explanation='The most representative value for olive oil is 3.29 kg CO2e per kg without considering biogenic C.', unit='kg CO2e per kg', result=3.29)]


## Generate final output

In [16]:
from typing import List
from estimator.prompt_templates.co2_search_prompts import CO2SearchResult
from estimator.prompt_templates.co2_sql_prompts import CO2Emissions
from estimator.prompt_templates.weight_est_prompts import WeightEstimates

negligeble_threshold = 0.01  # in kg


def generate_output(weight_estimates: WeightEstimates, co2_emissions: CO2Emissions, search_results: List[CO2SearchResult]) -> str:
    ingredients_output = []
    total_co2 = 0
    all_comments = []

    for weight_estimate in weight_estimates.weight_estimates:
        # Find corresponding co2 emission data and search result
        co2_data = next((item for item in co2_emissions.emissions if item.ingredient == weight_estimate.ingredient), None)
        search_result = next((item for item in search_results if item.ingredient == weight_estimate.ingredient), None)

        # Separate comments for clarity
        comments = {"Weight": weight_estimate.weight_calculation, "DB": co2_data.comment if co2_data else None, "Search": search_result.explanation if search_result else None}

        all_comments.append({"ingredient": weight_estimate.ingredient, "comments": comments})

        if weight_estimate.weight_in_kg is None:
            ingredients_output.append(f"{weight_estimate.ingredient}: unable to estimate weight")
            continue

        if weight_estimate.weight_in_kg <= negligeble_threshold:
            ingredients_output.append(f"{weight_estimate.ingredient}: weight on {round(weight_estimate.weight_in_kg,3)} kg is negligible")
            continue

        if co2_data and co2_data.co2_per_kg:
            co2_value = round(co2_data.co2_per_kg * weight_estimate.weight_in_kg, 2)
            ingredients_output.append(
                f"{weight_estimate.ingredient}: {round(co2_data.co2_per_kg, 2)} kg * {round(weight_estimate.weight_in_kg, 2)} kg CO2e / kg (DB) = {co2_value} kg CO2e"
            )
            total_co2 += co2_value

        elif search_result and search_result.result:
            co2_value = round(search_result.result * weight_estimate.weight_in_kg, 2)
            ingredients_output.append(
                f"{weight_estimate.ingredient}: {round(search_result.result, 2)} kg * {round(weight_estimate.weight_in_kg, 2)} kg CO2e / kg (Search) = {co2_value} kg CO2e"
            )
            total_co2 += co2_value

        else:
            ingredients_output.append(f"{weight_estimate.ingredient}: CO2e per kg ikke fundet")

    output = (
        "----------------------------------------"
        f"\nTotal CO2 emission: {round(total_co2,2)} kg CO2e"
        "\n----------------------------------------"
        "\nThe calculation method per ingredient is: X kg * Y kg CO2e / kg = Z kg CO2e"
    )
    output += "\n" + "\n".join(ingredients_output)
    output += "\n----------------------------------------"

    # Legends
    output += "\n\nLegends:"
    output += "\n(DB) - Data from SQL Database (https://denstoreklimadatabase.dk)"
    output += "\n(Search) - Data obtained from search"

    # Append comments
    output += "\n\nComments:"
    for comment in all_comments:
        ingredient = comment["ingredient"]
        output += f"\nFor {ingredient}:"
        for key, value in comment["comments"].items():
            if value:
                output += f"\n- {key}: {value}"

    return output


print(generate_output(parsed_output, parsed_sql_output, parsed_search_results))

----------------------------------------
Total CO2 emission: 2.97 kg CO2e
----------------------------------------
The calculation method per ingredient is: X kg * Y kg CO2e / kg = Z kg CO2e
150 g red lentils: 1.78 kg * 0.15 kg CO2e / kg (DB) = 0.27 kg CO2e
1 large onion: 0.9 kg * 0.28 kg CO2e / kg (DB) = 0.26 kg CO2e
1 pepper: 1.02 kg * 0.15 kg CO2e / kg (DB) = 0.15 kg CO2e
2 cubes of vegetable bouillon: weight on 0.008 kg is negligible
3 cloves garlic, pressed: 1.33 kg * 0.01 kg CO2e / kg (DB) = 0.02 kg CO2e
1 can of chopped tomatoes: 1.26 kg * 0.4 kg CO2e / kg (DB) = 0.5 kg CO2e
1 tin of tomato concentrate (140 g): 2.48 kg * 0.14 kg CO2e / kg (DB) = 0.35 kg CO2e
1 can of coconut milk (400 ml): 3.5 kg * 0.4 kg CO2e / kg (DB) = 1.4 kg CO2e
1 tbsp. lemon juice: 0.94 kg * 0.01 kg CO2e / kg (DB) = 0.01 kg CO2e
1 tsp. sugar: weight on 0.005 kg is negligible
½ teaspoon chili powder: weight on 0.003 kg is negligible
½ teaspoon ground cumin: weight on 0.003 kg is negligible
Possibly. 1/4 tsp